In [8]:
import xarray as xr
import numpy as np
import os
from datetime import datetime


In [9]:
def makeAverage(date_list : list[str],
                out_path: str) -> xr.Dataset :
    """
    Create an average dataset for a given date.
    
    Parameters:
    date_list (list[str]): List of dates in the format 'YYYY-MM-DD'.
    out_path (str): Path to save the averaged datasets.
    
    Returns:
    xr.Dataset: The averaged dataset in input1.nc, input2.nc, and input3.nc files.
    """
    surface_list = []
    deep_list = []
    deeper_list = []
    
    # Compute average for each date in the list.
    for i in date_list :

        surface = xr.open_dataset(f"{i}_avg_init_states/input1.nc").chunk(200)
        deep = xr.open_dataset(f"{i}_avg_init_states/input2.nc").chunk(200)
        deeper = xr.open_dataset(f"{i}_avg_init_states/input3.nc").chunk(200)

        surface_list.append(surface)
        deep_list.append(deep)
        deeper_list.append(deeper)

    a1 = xr.concat(surface_list, dim="time").mean(dim="time", keepdims=True).compute()
    a2 = xr.concat(deep_list, dim="time").mean(dim="time", keepdims=True).compute()
    a3 = xr.concat(deeper_list, dim="time").mean(dim="time", keepdims=True).compute()

    # Assign the time coordinate from the first surface dataset.
    coord_time = surface_list[0].time.values[0]
    a1 = a1.assign_coords(time=[coord_time])
    a2 = a2.assign_coords(time=[coord_time])
    a3 = a3.assign_coords(time=[coord_time])
    
    # Concatenate with the last date data in order to make GLONET input.

    input1 = xr.concat([a1, surface_list[-1]], dim="time").compute()
    input2 = xr.concat([a2, deep_list[-1]], dim="time").compute()
    input3 = xr.concat([a3, deeper_list[-1]], dim="time").compute()

    os.makedirs(out_path, exist_ok=True)
    input1.to_netcdf(f"{out_path}/input1.nc")
    input2.to_netcdf(f"{out_path}/input2.nc")
    input3.to_netcdf(f"{out_path}/input3.nc")

    return input1, input2, input3

In [ ]:
time_list = ["2025-04-01", "2025-05-01", "2025-06-01"]

a, b, c = makeAverage(time_list, "2025-04-to-2025-06_init_states")

In [11]:
a

<xarray.Dataset> Size: 39MB
Dimensions:  (time: 2, ch: 5, lat: 672, lon: 1440)
Coordinates:
  * ch       (ch) int64 40B 0 1 2 3 4
  * lat      (lat) float64 5kB -78.0 -77.75 -77.5 -77.25 ... 89.25 89.5 89.75
  * lon      (lon) float64 12kB -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * time     (time) datetime64[ns] 16B 2025-04-01 2025-06-01
Data variables:
    data     (time, ch, lat, lon) float32 39MB nan nan nan ... 0.04672 0.04662

In [ ]:
six_months = ["2025-01-01", "2025-02-01", "2025-03-01", "2025-04-01", "2025-05-01", "2025-06-01"]

e, f, g = makeAverage(six_months, "2025-01-to-2025-06_avg_init_states")

In [13]:
year = ["2024-07-01", "2024-08-01", "2024-09-01", "2024-10-01", "2024-11-01", "2024-12-01",
        "2025-01-01", "2025-02-01", "2025-03-01", "2025-04-01", "2025-05-01", "2025-06-01"]

h, i, j = makeAverage(year, "2024-07-to-2025-06_avg_init_states")